In [1]:
!pip install faiss-cpu faiss-gpu

In [2]:
# !pip install openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 36.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.0 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803373 sha256=cc39cea5409f902541aa9c846539cf9b6541210b3300f5030932eb0e01e55cf6
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [2]:
!apt-get install -y ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [3]:
!pip install ffmpeg-python


In [4]:
! pip install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --yes
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-21obt6xt
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-21obt6xt
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [5]:
import os
import pickle
import numpy as np
import cv2
from PIL import Image
from faiss import IndexFlatL2
# from transformers import (
#     Blip2Processor,
#     Blip2ForConditionalGeneration,
#     AutoTokenizer,
#     BitsAndBytesConfig
# )
from transformers import AutoProcessor, BlipForQuestionAnswering
#from transformers import BridgeTowerProcessor, BridgeTowerForContrastiveLearning
# from sentence_transformers import SentenceTransformer
#import whisper
import torch
import clip

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
# Set device
model, preprocess = clip.load("ViT-B/32", device=device)

In [8]:
def extract_frames(video_path, output_dir):
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_rate = int(cap.get(cv2.CAP_PROP_FPS))
    count = 0
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:  # Extract one frame per second
            frame_path = os.path.join(output_dir, f"frame_{count}.jpg")
            cv2.imwrite(frame_path, frame)
            frames.append(frame_path)
        count += 1
    cap.release()
    return frames

In [9]:
frames = extract_frames("input_video.mp4","video_frames")

In [ ]:
# frame_embeddings = []

# for frame_path in frames:
#     image = preprocess(Image.open(frame_path)).unsqueeze(0).to(device)
#     with torch.no_grad():
#         frame_embedding = model.encode_image(image).cpu().numpy()
#         frame_embeddings.append(frame_embedding)

# frame_embeddings = np.vstack(frame_embeddings)  # Shape: (num_frames, embedding_dim)

In [10]:
import faiss
# Assuming `preprocess`, `model`, `device`, and `frames` are already defined
frame_embeddings = []

# Generate embeddings for each frame
for frame_path in frames:
    image = preprocess(Image.open(frame_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        frame_embedding = model.encode_image(image).cpu().numpy()
        frame_embeddings.append(frame_embedding)

frame_embeddings = np.vstack(frame_embeddings)  # Shape: (num_frames, embedding_dim)

# Define dummy transcriptions
transcriptions = [f"Transcription for {frame_path}" for frame_path in frames]

# Initialize FAISS index
embedding_dim = frame_embeddings.shape[1]  # Embedding dimensionality
index = faiss.IndexFlatL2(embedding_dim)  # L2 distance metric
index.add(frame_embeddings)  # Add embeddings to the FAISS index

# Store metadata
metadata = [
    {"frame_path": frames[i], "transcription": transcriptions[i]} for i in range(len(frames))
]

# Save metadata to a file for persistence
with open("metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)

# Save the FAISS index
faiss.write_index(index, "faiss_index.bin")

print("Embeddings and metadata saved successfully!")


Embeddings and metadata saved successfully!


In [ ]:
# from sklearn.metrics.pairwise import cosine_similarity
# # Load FAISS index and metadata
# index = faiss.read_index("faiss_index.bin")
# with open("metadata.pkl", "rb") as f:
#     metadata = pickle.load(f)

# # Get user input
# user_prompt = input("Enter your query: ")

# # Tokenize and encode the user prompt into an embedding
# text = clip.tokenize([user_prompt]).to(device)
# with torch.no_grad():
#     text_embedding = model.encode_text(text).cpu().numpy()  # Shape: (1, embedding_dim)

# # Retrieve all frame embeddings from the FAISS index
# frame_embeddings = index.reconstruct_n(0, index.ntotal)  # Retrieve all stored embeddings

# # Compute cosine similarities
# similarities = cosine_similarity(text_embedding, frame_embeddings)  # Shape: (1, num_frames)
# best_frame_idx = similarities.argmax()  # Index of the most similar frame

# # Retrieve the corresponding metadata
# best_metadata = metadata[best_frame_idx]

# # Output results
# print("\nBest Match:")
# print(f"Frame Path: {best_metadata['frame_path']}")
# print(f"Transcription: {best_metadata['transcription']}")

Enter your query: hand

Best Match:
Frame Path: video_frames/frame_118.jpg
Transcription: Transcription for video_frames/frame_118.jpg


In [11]:
import torch
# from transformers import Blip2Processor, Blip2ForConditionalGeneration
from PIL import Image
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import faiss
import pickle
import logging
import time

In [12]:
logger = logging.getLogger('LLava_Chat')

In [13]:
!pip install accelerate bitsandbytes


In [19]:
from transformers import pipeline
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

model_id = "llava-hf/llava-1.5-7b-hf"

pipe = pipeline("image-text-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})
# pipe = pipeline("image-to-text", model=model_id)

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

Device set to use cuda:0


In [31]:
# import torch
# from transformers import AutoProcessor, LlavaForConditionalGeneration
# from PIL import Image

# # Load the model and processor, no need to manually move the model to the device
# model = LlavaForConditionalGeneration.from_pretrained(
#     "llava-hf/llava-1.5-7b-hf",
#     torch_dtype=torch.float16,
#     device_map="auto"
# )

# processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

# # Define your prompts
# prompts = [
#     "USER: <image>\nWho is in the video? ASSISTANT:",
# ]

# # Load the image
# image1 = Image.open("/content/video_frames/frame_0.jpg")

# # Process the input image and prompt
# inputs = processor(prompts, image=image1, padding=True, return_tensors="pt")

# # Generate the output
# output = model.generate(**inputs, max_new_tokens=20)

# # Print the result
# print(output)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:2137: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


KeyboardInterrupt: 

In [34]:
from transformers import CLIPProcessor, CLIPModel

class LLAVAChatModel:
    def __init__(self, llava_processor, llava_model, clip_processor, clip_model, device="cuda"):
        print(f"Initializing LLAVAChatModel on device: {device}")
        self.device = device

        # Load LLAVA
        print("Loading LLAVA processor...")
        self.llava_processor = llava_processor

        print("Loading LLAVA model...")
        self.llava_model = llava_model.to(self.device)

        # Load CLIP
        print("Loading CLIP processor...")
        self.clip_processor = clip_processor

        print("Loading CLIP model...")
        self.clip_model = clip_model.to(self.device)

        print("Model initialization complete!")

    def process_query(self, user_prompt, chat_history=None):
        try:
            print("\n=== Starting Query Processing ===")
            print(f"Processing query: {user_prompt}")

            # Load FAISS index and metadata
            print("Step 1: Loading FAISS index and metadata...")
            index = faiss.read_index("faiss_index.bin")
            with open("metadata.pkl", "rb") as f:
                metadata = pickle.load(f)
            print(f"Loaded index with {index.ntotal} frames")

            # Encode the user prompt using CLIP
            print("Step 2: Encoding user prompt with CLIP...")
            inputs = self.clip_processor(text=[user_prompt], return_tensors="pt").to(self.device)
            with torch.no_grad():
                text_embedding = self.clip_model.get_text_features(**inputs).cpu().numpy()
            print(f"Prompt encoded successfully. Embedding shape: {text_embedding.shape}")

            # Find the most similar frame using cosine similarity
            print("Step 3: Finding most similar frame...")
            similarities = cosine_similarity(text_embedding, index.reconstruct_n(0, index.ntotal))
            best_frame_idx = similarities.argmax()

            # Get metadata for the best frame
            best_metadata = metadata[best_frame_idx]
            frame_path = best_metadata['frame_path']
            transcription = best_metadata['transcription']
            print(f"Best matching frame found: {frame_path}")
            print(f"Transcription: {transcription}")

            # Generate response using LLAVA
            print("Step 4: Generating response with LLAVA...")
            chat_history = chat_history or []
            response = self.generate_response(
                frame_path=frame_path,
                user_prompt=user_prompt,
                chat_history=chat_history,
                transcription=transcription
            )
            print("Response generated successfully")

            return {
                'response': response,
                'frame_path': frame_path,
                'transcription': transcription,
                'similarity_score': similarities[0][best_frame_idx]
            }

        except Exception as e:
            print(f"Error in process_query: {str(e)}")
            import traceback
            print(f"Traceback: {traceback.format_exc()}")
            raise

    def generate_response(self, frame_path, user_prompt, chat_history, transcription):
        """Generate a response using LLAVA."""
        print("Loading and processing frame...")
        image = Image.open(frame_path)
        print("Formatting context...")
        formatted_history = "\n".join(
            f"{message['role'].capitalize()}: {message['content']}" for message in chat_history
        )
        prompt = f"""
        Chat History:
        {formatted_history}

        Frame Transcription:
        {transcription}

        Question: {user_prompt}
        Answer:"""
        print("Processing inputs for LLAVA...")
        inputs = self.llava_processor(
            text=[prompt], images=image, return_tensors="pt", padding=True
        ).to(self.device)
        print("Generating response...")
        output = self.llava_model.generate(**inputs, max_new_tokens=200)
        response = self.llava_processor.tokenizer.decode(output[0], skip_special_tokens=True)
        return response


In [ ]:
def main():
    print("Starting chat application...")

    # Initialize LLAVA processor and model
    processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
    model = LlavaForConditionalGeneration.from_pretrained(
        "llava-hf/llava-1.5-7b-hf",
        torch_dtype=torch.float16,
        device_map="auto"
    )

    # Initialize the chat model
    chat_model = LLAVAChatModel(processor, model)

    # Initialize chat history
    chat_history = []

    print("Chat model ready! You can start chatting...")

    while True:
        try:
            user_prompt = input("\nYou: ")
            if user_prompt.lower() in ['quit', 'exit']:
                print("Ending chat session...")
                break

            result = chat_model.process_query(user_prompt, chat_history)

            chat_history.append({'role': 'USER', 'content': user_prompt})
            chat_history.append({'role': 'ASSISTANT', 'content': result['response']})

            print("\nAssistant:", result['response'])
            print(f"Debug Info: Frame - {result['frame_path']}, Transcription - {result['transcription']}")
        except Exception as e:
            print(f"An error occurred: {str(e)}")


if __name__ == "__main__":
    main()